In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [9]:
## clarity database connection
import yaml
from sqlalchemy import create_engine

with open('clarity_creds.yaml') as credsfile:
    clarity_creds = yaml.safe_load(credsfile)


def createMSSQLconnection(database='clarity_snapshot_db', domain='UPHS', port='1433', timeout=3000):
    password = clarity_creds['password']
    username = clarity_creds['user']
    host = clarity_creds['server']
    user=domain+'\\'+username
    return create_engine('mssql+pymssql://{}:{}@{}:{}/{}?timeout={}'.format(user, password, host, port, database, timeout))
 
def get_res_dict(q, conn):
    res = conn.execute(q)
    data = res.fetchall()
    data_d =  [dict(zip(res.keys(), r)) for r in data]
    return data_d
 
def SQLquery2df(q, conn):
    return pd.DataFrame(get_res_dict(q, conn))
    
conn = createMSSQLconnection()
print(conn.execute("select @@VERSION").fetchall()[0][0])

Microsoft SQL Server 2016 (SP1-CU7-GDR) (KB4057119) - 13.0.4466.4 (X64) 
	Dec 22 2017 11:25:00 
	Copyright (c) Microsoft Corporation
	Enterprise Edition: Core-based Licensing (64-bit) on Windows Server 2012 R2 Standard 6.3 <X64> (Build 9600: ) (Hypervisor)



In [12]:
q = """
-- MEWS
select
  FLO_MEAS_NAME,
  pat.PAT_MRN_ID,
  pat.BIRTH_DATE,
  peh.PAT_ENC_CSN_ID,
  peh.HOSP_ADMSN_TIME,
  peh.HOSP_DISCH_TIME,
  peh.INP_ADM_DATE,
  dep.DEPARTMENT_NAME,
  dep.DEPT_ABBREVIATION,
  RECORDED_TIME,
  ENTRY_TIME,
  TAKEN_USER_ID,
  ENTRY_USER_ID,
  MEAS_VALUE,
  MEAS_COMMENT,
  ISACCEPTED_YN,
  NEEDS_COSIGN_C,
  FLT_ID,
  FLO_DAT_USED,
  meas.UPDATE_DATE,
  FLO_CNCT_DATE_REAL,
  DOCUMENTATION_SOURCE_C,
  dat.FLO_MEAS_ID,
  DISP_NAME,
  ROW_TYP_C,
  VAL_TYPE_C,
  UNITS,
  DUPLICATABLE_YN
  FROM IP_FLWSHT_MEAS meas
  LEFT JOIN IP_FLO_GP_DATA dat
    ON dat.FLO_MEAS_ID = meas.FLO_MEAS_ID  -- and  UPPER(dat.FLO_MEAS_NAME) like '%RIGHTCARE%'
    INNER JOIN IP_FLWSHT_REC rec
        ON rec.FSD_ID = meas.FSD_ID
    INNER JOIN PAT_ENC_HSP peh
        ON rec.INPATIENT_DATA_ID = peh.INPATIENT_DATA_ID
    INNER JOIN PATIENT pat
        ON peh.PAT_ID = pat.PAT_ID
    INNER JOIN CLARITY_DEP dep on peh.DEPARTMENT_ID = dep.DEPARTMENT_ID
WHERE meas.ENTRY_TIME > CAST('2020-06-10' AS DATE)
AND INP_ADM_DATE >  CAST('2020-06-01' AS DATE)
AND dat.DISP_NAME in ('Resp',
           'Pulse',
           'Temp',
           'BP',
           'Level of Consciousness',
           'Level of consciousness',
           'Altered Level of Consciousness',
           'Urine Output (mL)',
           'Calculated Urine (mL)',
           'Urine');
"""


mews = SQLquery2df(q,conn)
mews.shape

(433597, 28)